# Project : Instagram Data Clustering

# 1. Data Introduce
### 1.1 Purpose : To find out popular hashtags are being used and analyze topic using Latent Dirichlet Allocation.

### 1.2 Data set:
   > - Train Data : 43264 rows, 20 columns

In [1]:
from IPython.display import display, Markdown
import private_function as pf
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pickle as pkl
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

from sklearn.model_selection import GridSearchCV

# Pretty display for notebooks
%matplotlib inline

# Ignore the warnings
import warnings
warnings.filterwarnings('ignore')

def get_x_train_and_features_name(df):
    cnt_vectorizer = CountVectorizer(lowercase=False)
    X_train = cnt_vectorizer.fit_transform(df)
#     cnt_feature_names = cnt_vectorizer.get_feature_names()
    return X_train

def run_lda(df, n_topic, max_iter = 100):
    cnt_vectorizer = CountVectorizer(lowercase=False)
    X_train = cnt_vectorizer.fit_transform(df)
    cnt_feature_names = cnt_vectorizer.get_feature_names()

    # hyper parameter
    alpha = 1
    beta = 1

    # train the model
    lda = LatentDirichletAllocation(n_components=n_topic, doc_topic_prior=alpha,\
                                    topic_word_prior=beta, learning_method='online', max_iter=max_iter)

    %time lda.fit_transform(X_train)
    
    return lda, cnt_vectorizer, cnt_feature_names

def model_test(test_list, lda_model, cnt_vectorizer):
    test_ = cnt_vectorizer.transform(test_list)
    doc_topic_dist_unnormalized = np.matrix(lda_model.transform(test_))
    doc_topic_dist = doc_topic_dist_unnormalized/doc_topic_dist_unnormalized.sum(axis=1)
    print(doc_topic_dist.argmax(axis=1))
    return doc_topic_dist.argmax(axis=1)

def get_topic_using_lda(lda_model, x_train):
    doc_topic_dist_unnormalized = np.matrix(lda_model.transform(x_train))
    # normalize the distribution (only needed if you want to work with the probabilities)
    doc_topic_dist = doc_topic_dist_unnormalized/doc_topic_dist_unnormalized.sum(axis=1)
    return doc_topic_dist.argmax(axis=1)

df_train = pkl.load(open("asset/df_train.pkl", "rb"))

In [26]:
df_li = pf.getAllDataFrame()
train = pf.makeOneTrainDf(df_li)
train = pf.make_df_i_want(train)
counter = pf.get_counter(train)
counter.most_common()
tag_count = 0
for tup in counter.most_common():
    tag_count += tup[1]
tag_count, len(counter)

['직장인', '럽스타그램', '서울맛집', '먹스타그램', '친스타그램', '여행', '일상', '셀스타그램']

['seoul']

job.csv : 2213

insta_new_5.csv : 2136

insta_new_4.csv : 2148

...

seoul.csv : 880

yongin.csv : 960

gyeonggido.csv : 960

29
Df list length : 29
Df length changes after concat (if 0 means all datas are unique) : 0
Df length changes after concat (if 0 means all datas are unique) : 0
Df length changes after concat (if 0 means all datas are unique) : 1095
Df length changes after concat (if 0 means all datas are unique) : 411
Df length changes after concat (if 0 means all datas are unique) : 54
Df length changes after concat (if 0 means all datas are unique) : 131
Df length changes after concat (if 0 means all datas are unique) : 0
Df length changes after concat (if 0 means all datas are unique) : 35
Df length changes after concat (if 0 means all datas are unique) : 16
Df length changes after concat (if 0 means all datas are unique) : 75
Df length changes after concat (if 0 means all datas are unique) : 11
Df length changes after concat (if 0 means all datas are unique) : 42
Df length changes after concat (if 0 means all datas are unique) : 358
Df length changes after concat (if 0 means all datas are unique) 

(46899, 14241)

## Topic별 Top 20 키워드

### Topic 0: 여행스타그램
여행에미치다 여행스타그램 여행 travel 휴가 korea photography 일본 photo seoul 바다 감성사진 한국 trip 풍경 웨딩 럽스타그램 스냅 예신 결혼

### Topic 1: 대출 광고
신용카드현금화 카드깡 상품권현금화 소액결제 소액결제현금화 모바일문화상품권 굿핀 해피머니 컬쳐랜드 신용카드대출 일수대출 모바 홍대맛집 휴대폰소액결제 차스타그램 핸드폰소액결제 빈티지 골프 상품권 비트코인

### Topic 2: 운동스타그램
다이어트 운동하는여자 다이어터 다이어트식단 용인 diet 다이어트그램 꽃다발 꽃스타그램 식단 헬스 flower 운동 운스타그램 fitness 플로리스트 유지어터 workout 운동하는남자 식단일기

### Topic 3: 신발 광고
축구 헬스 커플신발 푸마 레플 해외직구 아웃도어 커플운동화 나이키에어맥스 신발쇼핑몰 신상신발 등산화 신발도매 명품등산화 유행신발 아디다스신발 GGDB 아식스 아디다스울트라부스트 NEWBALANCE

### Topic 4: Kpop
kpop bts exo 귀여운 blackpink 여자 korea kawaii 아름다운 自撮り boy asmr korean 속초맛집 Asia jisoo kpopl4l tomboy селфи 自分撮り

### Topic 5: 서울 맛집
서울맛집 부산맛집 강남맛집 대구맛집 곱창 홍대맛집 강남역 강남역맛집 대전맛집 강남 맛스타그램 역삼맛집 이태원맛집 역삼동맛집 신논현맛집 대치동맛집 서면맛집 곱창맛집 역삼동 서초맛집

### Topic 6: 럽스타그램
남친이랑 데이트 커플 사랑해 럽스타 커플스타그램 남자친구 영화 연애중 연애 행복 남친 고마워 사랑꾼 ㅋㅋㅋ 행복해 데이트그램 사랑 여자친구 화이팅

### Topic 7: 육아스타그램
럽스타그램 육아 육아스타그램 육아소통 육아맘 맘스타그램 도치맘 일상 젊줌마 딸스타그램 인스타베이비 애스타그램 직장인 아들스타그램 사랑해 아들맘 세젤귀 반려견 워킹맘 멍스타그램

### Topic 8: 토토 등 사이트 및 명품 광고
토토사이트추천 겐조 샤넬 서울맛집 운동하는남자 럽스타 핫플레이스 커플 구찌 에르메스 술스타그램 토토사이트 발렌티노 팔찌 스포츠가족방 사설사이트추천 파워볼가족방 프로토가족방 픽스터가족방 사설사이트

### Topic 9: 폰케이스 광고
선물 직장인 아이폰케이스 냥스타그램 마카롱 취미 고양이 폰케이스 아이폰8 커플케이스 공감 그림 글스타그램 기념일 디저트 일러스트 아이폰7케이스 아이폰x케이스 글귀 귀걸이

### Topic 10: 먹스타그램
먹스타그램 먹방 맛스타그램 맛집 맥주스타그램 food 술스타그램 점심 먹스타 맥주 instafood foodstagram 먹방스타그램 음식 존맛 맛스타 먹부림 푸드스타그램 맛있다 디저트

### Topic 11: 일상적인 해시태그
일상 맞팔 데일리 소통 선팔 셀스타그램 좋아요 셀카 셀피 팔로우 먹스타그램 daily 얼스타그램 일상스타그램 선팔하면맞팔 오오티디 럽스타그램 인친 좋아요반사 ootd

### Topic 12: 제주도 맛집 광고
제주도 제주도맛집 제주맛집 제주 서귀포맛집 제주여행 f4follow 선팔맞팔 제주맛집추천 소통해요 소통하자 제주도그램 제주서귀포맛집 맥주스타그램 제주도여행 제주도흑돼지맛집 제주흑돼지 제주도흑돼지 서귀포흑돼지맛집 서귀포흑돼지

### Topic 13: 거제도 맛집?
거제도맛집 거제맛집 포천여행 짖어야개다 평일 포천카페 햄스터 포천이동갈비 삼척맛집 성남애견미용 근무시간 스킨케어 포천맛집 거제대명리조트맛집 성남애견호텔 골든햄스터 살롱순라 햄스타그램 2018년 삼척여행

### Topic 14: 반려견스타그램 (lda topic number = 12를 통해서 찾음, 해당 모델에선 topic 8)
반려견 멍스타그램 강아지 개스타그램 냥스타그램 아이폰케이스 dog 댕댕이 고양이 펫스타그램 폰케이스 견스타그램 아이폰8 커플케이스 cat dogstagram 아이폰7케이스 아이폰x케이스 puppy 독스타그램

### Topic 15: 맥주스타그램 및 치맥 (lda topic number = 13을 통해서 찾음, 해당 모델에선 topic 6)
맥주스타그램 술스타그램 맥주 beer 혼술 치맥 수제맥주 소주 소맥 치킨 맥주한잔 술집 beerstagram 크래프트비어 맥주그램 존맛탱 낮술 생맥주 술스타 craftbeer

### 1, 3, 8, 9, 12는 다 광고성 글이다

In [3]:
tmp_li = [1, 3, 8, 9, 12]
count = 0
for topic in tmp_li:
    length = len(df_train[df_train.target == topic])
    count += length
    print(topic, length)
count

1 570
3 847
8 1451
9 1114
12 529


4511

In [4]:
df_train.columns

Index(['caption', 'comment_cnt', 'first_comment', 'id', 'is_video', 'likes',
       'loc_id', 'loc_lat', 'loc_lon', 'loc_name', 'owner_id', 'owner_name',
       'shortcode', 'taken_at_timestamp', 'video_view_count', 'tags',
       'tags_cnt', 'caption_only', 'tags_str', 'duplicated_tag', 'topic_type',
       'topic_type_13', 'topic_type_12', 'target'],
      dtype='object')

In [5]:
columns = ['caption_only', 'tags_str', 'duplicated_tag', 'topic_type', 'topic_type_13', 'topic_type_12', 'target']

In [29]:
test = train[train.tags_cnt == 0]
df_test = test.caption

In [30]:
train = df_train[columns]

In [31]:
train["caption_only"] = train["caption_only"].apply(lambda a: "" if type(a) == float else a)

In [32]:
train["caption_only"] = train["caption_only"].apply(lambda a: a.strip())

In [33]:
train = train[train["caption_only"] != ""]

In [69]:
tfidf_max_df_params = np.arange(0.8, 1, 0.05)
# tfidf_min_df_params = np.arange(0, 0.3, 0.05)
tfidf_max_df_params

array([0.8 , 0.85, 0.9 , 0.95])

In [70]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB(alpha=0.01)) 
])

parameters = {
    'tfidf__max_df': tfidf_max_df_params,
#     'tfidf__min_df': tfidf_min_df_params,
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)]
}

grid_search_tune = GridSearchCV(pipeline, parameters, cv=10, n_jobs=3, verbose=3)

# %%time
grid_search_tune.fit(X_train, y_train)

# print("Best parameters set:")
# print(grid_search_tune.best_estimator_.steps)

Fitting 10 folds for each of 12 candidates, totalling 120 fits
[CV] tfidf__max_df=0.8, tfidf__ngram_range=(1, 1) ....................
[CV] tfidf__max_df=0.8, tfidf__ngram_range=(1, 1) ....................
[CV] tfidf__max_df=0.8, tfidf__ngram_range=(1, 1) ....................
[CV]  tfidf__max_df=0.8, tfidf__ngram_range=(1, 1), score=0.604261796042618, total=   1.2s
[CV]  tfidf__max_df=0.8, tfidf__ngram_range=(1, 1), score=0.5921413341455986, total=   1.2s
[CV] tfidf__max_df=0.8, tfidf__ngram_range=(1, 1) ....................
[CV]  tfidf__max_df=0.8, tfidf__ngram_range=(1, 1), score=0.6093845216331505, total=   1.2s
[CV] tfidf__max_df=0.8, tfidf__ngram_range=(1, 1) ....................
[CV] tfidf__max_df=0.8, tfidf__ngram_range=(1, 1) ....................
[CV]  tfidf__max_df=0.8, tfidf__ngram_range=(1, 1), score=0.5973788479122218, total=   1.2s
[CV]  tfidf__max_df=0.8, tfidf__ngram_range=(1, 1), score=0.6022554099359951, total=   1.2s
[CV] tfidf__max_df=0.8, tfidf__ngram_range=(1, 1) ..

[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:   39.4s


[CV]  tfidf__max_df=0.8, tfidf__ngram_range=(1, 3), score=0.6187862153095456, total=   5.4s
[CV] tfidf__max_df=0.8, tfidf__ngram_range=(1, 3) ....................
[CV]  tfidf__max_df=0.8, tfidf__ngram_range=(1, 3), score=0.5979255643685174, total=   5.3s
[CV] tfidf__max_df=0.8500000000000001, tfidf__ngram_range=(1, 1) .....
[CV]  tfidf__max_df=0.8500000000000001, tfidf__ngram_range=(1, 1), score=0.604261796042618, total=   1.3s
[CV] tfidf__max_df=0.8500000000000001, tfidf__ngram_range=(1, 1) .....
[CV]  tfidf__max_df=0.8, tfidf__ngram_range=(1, 3), score=0.6058015267175573, total=   5.3s
[CV] tfidf__max_df=0.8500000000000001, tfidf__ngram_range=(1, 1) .....
[CV]  tfidf__max_df=0.8500000000000001, tfidf__ngram_range=(1, 1), score=0.5921413341455986, total=   1.3s
[CV] tfidf__max_df=0.8500000000000001, tfidf__ngram_range=(1, 1) .....
[CV]  tfidf__max_df=0.8500000000000001, tfidf__ngram_range=(1, 1), score=0.6093845216331505, total=   1.3s
[CV] tfidf__max_df=0.8500000000000001, tfidf__ngr

[CV] tfidf__max_df=0.9000000000000001, tfidf__ngram_range=(1, 2) .....
[CV]  tfidf__max_df=0.9000000000000001, tfidf__ngram_range=(1, 2), score=0.6080463273392258, total=   3.3s
[CV] tfidf__max_df=0.9000000000000001, tfidf__ngram_range=(1, 2) .....
[CV]  tfidf__max_df=0.9000000000000001, tfidf__ngram_range=(1, 2), score=0.6083511124657117, total=   3.3s
[CV] tfidf__max_df=0.9000000000000001, tfidf__ngram_range=(1, 2) .....
[CV]  tfidf__max_df=0.9000000000000001, tfidf__ngram_range=(1, 2), score=0.6048780487804878, total=   3.2s
[CV] tfidf__max_df=0.9000000000000001, tfidf__ngram_range=(1, 2) .....
[CV]  tfidf__max_df=0.9000000000000001, tfidf__ngram_range=(1, 2), score=0.6181762732540409, total=   3.1s
[CV] tfidf__max_df=0.9000000000000001, tfidf__ngram_range=(1, 2) .....
[CV]  tfidf__max_df=0.9000000000000001, tfidf__ngram_range=(1, 2), score=0.5933496034167175, total=   3.0s
[CV] tfidf__max_df=0.9000000000000001, tfidf__ngram_range=(1, 3) .....
[CV]  tfidf__max_df=0.9000000000000001,

[Parallel(n_jobs=3)]: Done 120 out of 120 | elapsed:  3.1min finished


GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...ue,
        vocabulary=None)), ('clf', MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True))]),
       fit_params=None, iid=True, n_jobs=3,
       param_grid={'tfidf__max_df': array([0.8 , 0.85, 0.9 , 0.95]), 'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [72]:
grid_search_tune.best_params_, grid_search_tune.best_score_

({'tfidf__max_df': 0.8, 'tfidf__ngram_range': (1, 3)}, 0.6118429076716673)

> parameters = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__estimator__alpha': (1e-2, 1e-3)
}

##### 패러미터 성적

In [50]:
grid_search_tune.best_params_

{'clf__alpha': 0.01, 'tfidf__max_df': 0.25, 'tfidf__ngram_range': (1, 3)}

In [49]:
grid_search_tune.best_score_

0.6118429076716673

In [78]:
X_train, X_test, y_train, y_test = train_test_split(train.caption_only, train.topic_type, test_size=0.1, random_state=1)
len(X_train), len(X_test), len(y_train), len(y_test)

(32796, 3645, 32796, 3645)

In [88]:
clf = Pipeline([ 
    ('vect', TfidfVectorizer(ngram_range=(1, 3))), 
    ('clf', MultinomialNB(alpha=0.01)) 
])

In [95]:
%%time
model_all_data = clf.fit(train.caption_only, train.topic_type)

CPU times: user 5.34 s, sys: 328 ms, total: 5.66 s
Wall time: 5.67 s


In [166]:
test_list = [
    "방탄소년단 bts 최고",
    "배고파.. 맥주먹고싶다",
    "올만에 데이트!!!",
    "사랑",
    "행복한 하루!! 날씨가 너무 좋아서 여행을 떠나고 싶은 하루였어!!",
    "대출은",
    "어이없넹",
    "밥",
    "차이가 ㅇ벗너",
    "폰케이스",
    "어이가 없네",
    "똑같너",
    "성능이 너무 구리다!!!! 미치겠다"
]

In [128]:
model_all_data.predict(test_list)

array([11, 11,  6, 11, 11, 11, 11, 11, 11, 11,  7, 11,  7])

In [129]:
model.predict(test_list)

array([11, 11,  6, 11, 11, 11, 11, 11, 11, 11,  7, 11,  7])

In [96]:
y_pred_used_all_data = model_all_data.predict(X_test)

In [98]:
accuracy_score(y_test, y_pred_used_all_data) 

0.9775034293552812

In [89]:
%%time
model = clf.fit(X_train, y_train)

CPU times: user 4.8 s, sys: 283 ms, total: 5.09 s
Wall time: 5.12 s


In [90]:
y_pred = model.predict(X_test)

In [93]:
y_pred_train = model.predict(X_train)

In [94]:
accuracy_score(y_train, y_pred_train) 

0.9726186120258569

In [91]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.55      0.36      0.44       198
          1       0.96      0.72      0.82        67
          2       0.67      0.22      0.33        55
          3       0.40      0.08      0.13        75
          4       0.85      0.39      0.54        28
          5       0.78      0.69      0.73       143
          6       0.33      0.12      0.18       174
          7       0.53      0.45      0.49       343
          8       0.92      0.94      0.93       131
          9       0.76      0.42      0.55       153
         10       0.50      0.58      0.54       573
         11       0.65      0.80      0.72      1632
         12       0.92      0.59      0.72        58
         13       0.89      0.53      0.67        15

avg / total       0.62      0.63      0.61      3645



In [92]:
accuracy_score(y_test, y_pred) 

0.6279835390946502

### imbalance problem

In [133]:
from imblearn.under_sampling import *
from imblearn.over_sampling import *
from imblearn.combine import *

In [162]:
x_sample, y_sample = train.caption_only, train.topic_type

In [163]:
tf_vec = TfidfVectorizer(ngram_range=(1, 3))

In [164]:
x_sample = tf_vec.fit_transform(x_sample)

In [138]:
x_random, y_random = RandomUnderSampler(random_state=0).fit_sample(x_sample, y_sample)

In [140]:
x_sample.shape

(36441, 761430)

In [139]:
x_random.shape, y_random.shape

((1988, 761430), (1988,))

In [141]:
multimodel = MultinomialNB(alpha=0.01)

In [142]:
multimodel.fit(x_random, y_random)

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [171]:
X_test = tf_vec.transform(X_test)

In [173]:
y_pred = multimodel.predict(X_test)

In [178]:
y_pred

array([11,  5,  2, ...,  6,  0, 11])

In [174]:
accuracy_score(y_test, y_pred)

0.440877914951989

In [182]:
np.unique(y_pred)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])